# AssistantAgent

The `AssistantAgent` in AutoGen is a powerful agent for building multi-agent applications. It leverages a language model and can utilize various tools to accomplish tasks. 

### Code example
Here's an example demonstrating how to use `AssistantAgent` with a tool to get the current date and time:

In [ ]:
# import required libraries
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
import datetime

In [ ]:
# Define a tool to get the current date and time
async def get_current_datetime() -> str:
    """Gets the current date and time."""
    
    now = datetime.datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")

In [ ]:

# Create an AssistantAgent
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini"
)
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[get_current_datetime],
    system_message="Use tools to solve tasks.",
)

In [ ]:

response = await agent.on_messages(
    messages=[TextMessage(content="What is the current date and time?", source="user")],
    cancellation_token=CancellationToken(),
)
print("Response:", response)
print("Final Response:", response.chat_message)
print("Inner Messages:", response.inner_messages)

## Understanding the `on_messages()` Response

The `on_messages()` function serves as the primary communication channel with the agent. It returns a `Response` object, which acts as a container holding valuable information about the agent's processing.

Here's a breakdown of the key components within the `Response` object:

* **`chat_message`:**
    * This attribute stores the agent's final, conclusive response. It represents the ultimate answer or output the agent has generated based on the input message and its internal processing.
    * Think of it as the agent's final statement or conclusion.

* **`inner_messages`:**
    * This attribute contains a list of messages that document the agent's internal reasoning process.
    * It effectively reveals the agent's "thought process," showing the steps, considerations, and tool usages that led to the final `chat_message`.
    * By examining `inner_messages`, you can gain insights into how the agent arrived at its conclusion, allowing for debugging, analysis, and a deeper understanding of its behavior.

## Important Note: `on_messages()` and Agent State

**Understanding State Updates:**

It's crucial to understand that the `on_messages()` method modifies the agent's internal state. Specifically, each call to `on_messages()` appends the provided messages to the agent's message history.

**Implications:**

* **Avoid Redundant Calls:**
    * Repeatedly calling `on_messages()` with the same messages or the entire history will result in duplicate entries in the agent's history. This can lead to unexpected behavior and potentially degrade performance.
* **Use New Messages:**
    * To ensure proper functioning and accurate agent responses, always call `on_messages()` with *new* messages that represent the next step in the interaction. Do not send comple user-assistent interaction.
* **Maintaining Context:**
    * The agent uses its message history to maintain context during conversations. Therefore, avoid adding uneeded duplicate messages.

**In essence, treat `on_messages()` as a method that advances the agent's conversation and knowledge. Do not use it to replay or reiterate past interactions.**

## The `run()` Method: A Convenience Wrapper

In addition to `on_messages()`, you can utilize the `run()` method for interacting with the agent.

**Key Characteristics:**

* **Convenience Method:**
    * The `run()` method acts as a simplified, high-level interface that internally calls `on_messages()`. This provides a more streamlined way to execute agent interactions.
* **Teams Interface Compatibility:**
    * The `run()` method is designed to follow the same interface as the `Teams` object. This consistency simplifies integration and promotes a unified approach to agent interactions.
* **`TaskResult` Object:**
    * Instead of returning a `Response` object like `on_messages()`, `run()` returns a `TaskResult` object.
    * The `TaskResult` object provides a structured representation of the agent's execution, encapsulating the outcome and relevant details of the task.
    * This task result object can be used to more easily access the results of an agents actions.

In [ ]:
# Using run()
task_result = await agent.run(
    task=[TextMessage(content="What is the current date and time?", source="user")],
    cancellation_token=CancellationToken(),
)
print("Task Result:", task_result)
print("Final Response:", task_result.messages[-1].content)